In [2]:
# import modules
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext
import pandas as pd
import numpy as np


In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [24]:
# read the csv with library
df = sqlContext.read.format('com.databricks.spark.csv')\
					.options(header='true', inferSchema='true')\
					.load('C:/ssafy/2nd/s03p22a305/bigdata/basic/2019-09.csv')\
					.cache()


pyspark.sql.dataframe.DataFrame

In [25]:
df = df.selectExpr("'성별' as gender", "'연령' as age", "'승인시간대1' as time", "'가맹점업종코드' as code")

In [26]:
df.printSchema()
df.count()

# convert the df to tmp table (as if it's in database)
df.registerTempTable("df_tmp")

root
 |-- gender: string (nullable = false)
 |-- age: string (nullable = false)
 |-- time: string (nullable = false)
 |-- code: string (nullable = false)



In [29]:
# extract data from table with sql
df1 = sqlContext.sql("select * from df_tmp where time >=10")

# # other sql examples
# sqlContext.sql("select datetime, count(1) from df_tmp group by datetime order by datetime").show()
# sqlContext.sql("select count(distinct sessionid) as session_cnt from df_tmp where documentposition = 'MYPOLARISDRIVE' group by ext having count(distinct sessionid) ").show()

print(df.count())
print(df1.count())


1048575
0


In [28]:

## Lazy Execution
df2 = sqlContext.sql("select * from df_tmp")

df2_pdf = df2.select("sessionid", "ext").filter(" ext == 'PDF' or ext = 'DOC'").dropDuplicates().cache()
df2_pdf.distinct().count()


AnalysisException: cannot resolve '`sessionid`' given input columns: [df_tmp.age, df_tmp.code, df_tmp.gender, df_tmp.time];;
'Project ['sessionid, 'ext]
+- Project [gender#1466, age#1467, time#1468, code#1469]
   +- SubqueryAlias df_tmp
      +- Project [성별 AS gender#1466, 연령 AS age#1467, 승인시간대1 AS time#1468, 가맹점업종코드 AS code#1469]
         +- Relation[성별#1458,연령#1459,승인시간대1#1460,가맹점업종코드#1461] csv


In [8]:
df2_min_date = df2.groupby("sessionid").agg(min("datetime").alias("min_date"))
df2_min_date.show()

df2_join = df2_pdf.join(df2_min_date, "sessionid", "left")
df2_join.show()

df2_join1 = df2_join.groupby("min_date", "ext").agg(count("sessionid").alias("cnt"))

df2_join1.describe().show()

+--------------------+---------+
|           sessionid| min_date|
+--------------------+---------+
|000ad8bfdff1ac4ab...|2016.7.17|
|008e1cb0462257ec9...| 2016.7.3|
|013f6c2b225e3367d...|2016.7.13|
|01ef158e70ef4ce7b...| 2016.7.9|
|02bb1592965d0dd8c...|2016.7.20|
|03288dde777589053...| 2016.7.6|
|0376d4749a8d1d710...|2016.7.24|
|0378158b8aee19123...| 2016.7.1|
|045c6e82a181152ae...|2016.7.27|
|04917f8f10bf22dab...|2016.7.21|
|04e0324d6a64c743d...|2016.7.19|
|050a25d3b0bf9081e...|2016.7.20|
|057cd5d22a7ebcf2b...|2016.7.28|
|05b3df1764b2f8435...|2016.7.13|
|06554b488500b58ea...|2016.7.15|
|065f2825395cd914f...|2016.7.21|
|07bdb289d126fa929...| 2016.7.6|
|082e52a48bc92a8fa...|2016.7.29|
|08f38ff50c2401683...|2016.7.28|
|09a604c8fe3d4e3c9...|2016.7.25|
+--------------------+---------+
only showing top 20 rows

+--------------------+---+---------+
|           sessionid|ext| min_date|
+--------------------+---+---------+
|008e1cb0462257ec9...|PDF| 2016.7.3|
|013f6c2b225e3367d...|PDF|2016.7.1

In [9]:
# Pandas
df2_pd = df2.toPandas()
df2_pd.groupby("ext")['sessionid'].count().sort_values(ascending=False)
df2_pd['ext'].value_counts()

PDF      82004
DOCX     58303
XLSX     52118
HWP      26244
DOC      24586
XLS      24489
PPTX     15385
TXT       9814
PPT       5382
PPSX      2374
ODT        820
PPS        324
JPG          9
SHEET        5
WORD         2
PNG          2
Name: ext, dtype: int64

In [10]:
df2_pd

,actiontype,ismydoc,ext,sessionid,documentposition,datetime
0,OPEN,False,PDF,9400fd2e43d7dc2d054ca78806236ee1,LOCALSTORAGE,2016.7.18
1,CLOSE,False,PDF,9400fd2e43d7dc2d054ca78806236ee1,LOCALSTORAGE,2016.7.18
2,OPEN,True,PDF,9400fd2e43d7dc2d054ca78806236ee1,MYPOLARISDRIVE,2016.7.18
3,CLOSE,True,PDF,9400fd2e43d7dc2d054ca78806236ee1,MYPOLARISDRIVE,2016.7.18
4,OPEN,False,PDF,f191063c562691041dfa935ff0876975,OTHERAPP,2016.7.6
...,...,...,...,...,...,...
301856,CLOSE,True,XLSX,2ed068d5e6a72e80e4a997c01c59f782,OTHERAPP,2016.7.7
301857,OPEN,False,DOCX,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016.7.5
301858,RESET,True,DOCX,41b8c1df291edd2aec30fc610e5b676e,OTHERAPP,2016.7.5
301859,OPEN,False,PPT,9b5ee236571c2fcbe25132b6ba71d764,OTHERAPP,2016.7.14
